In [ ]:
import trimesh
import open3d as o3d
import numpy as np

In [ ]:
scene = trimesh.load('ascheberg_DA_ip.glb') # can change scene name
mesh = list(scene.geometry.values())[0]

In [ ]:
# REMOVING DEGENERATE/DUPLICATE FACES W TRIMESH
faces = mesh.faces
normals = mesh.face_normals
triangles = mesh.triangles
vertices = mesh.vertices
# unique_edges = mesh.edges_unique
# print("Unique edges:")
# print(unique_edges)

print(f"BEFORE: Mesh has {len(faces)} triangles.")

print("Removing degenerate/duplicate faces...")
# remove degenerate/duplicate faces from mesh
mesh.update_faces(mesh.nondegenerate_faces())
mesh.update_faces(mesh.unique_faces())

faces = mesh.faces
print(f"AFTER: Mesh has {len(faces)} triangles.")


In [ ]:
# MAKE THE OPEN3D MESH
o3d_mesh = o3d.geometry.TriangleMesh(
    o3d.utility.Vector3dVector(mesh.vertices),
    o3d.utility.Vector3iVector(mesh.faces)
)

In [ ]:
# MANIFOLD CHECK + VISUALIZATION

# check if edge manifold (no redundant triangles)
edge_manifold_status = o3d_mesh.is_edge_manifold(allow_boundary_edges=True)
print("Open3D edge_manifold check is: ", edge_manifold_status)

# extract non-manifold edges
edges = np.asarray(o3d_mesh.get_non_manifold_edges(allow_boundary_edges=True))
vertices = np.asarray(o3d_mesh.vertices)

print(edges)

# visualize non-manifold edges
lines_non_manifold = [[int(edge[0]), int(edge[1])] for edge in edges]
print(lines_non_manifold)

# create a LineSet for non-manifold edges
line_set_non_manifold = o3d.geometry.LineSet()
line_set_non_manifold.points = o3d.utility.Vector3dVector(vertices)
line_set_non_manifold.lines = o3d.utility.Vector2iVector(lines_non_manifold)
line_set_non_manifold.colors = o3d.utility.Vector3dVector(
    [[1, 0, 0]] * len(lines_non_manifold)  # Red for non-manifold edges
)

# create the original mesh for context
o3d_mesh.paint_uniform_color([0.7, 0.7, 0.7])  # Light gray for the mesh

# visualize the mesh and highlighted edges
o3d.visualization.draw_geometries([o3d_mesh, line_set_non_manifold])

In [ ]:
# REMOVE NON MANIFOLD EDGES
o3d_mesh_remove_non_manifold = o3d_mesh
o3d_mesh_remove_non_manifold.remove_non_manifold_edges()

o3d.visualization.draw_geometries([o3d_mesh_remove_non_manifold], mesh_show_wireframe=True)

In [ ]:
# MERGE CLOSE VERTICES + VISUALIZATION
o3d_mesh_merge_close_verts = o3d_mesh
print("# of vertices before merge: ", len(o3d_mesh_merge_close_verts.vertices))
o3d_mesh_merge_close_verts.merge_close_vertices(0.01)
print("# of vertices after merge: ", len(o3d_mesh_merge_close_verts.vertices))
o3d.visualization.draw_geometries([o3d_mesh_merge_close_verts], mesh_show_wireframe=True)
